# Brazil Export/Import of Jewelry - Analysis

On my spare time I am a silversmith. Since I plan on eventualy selling some pieces, any info on how the Brazilian jewelry market work is appreciated.

Brazil releases info from what was imported/exported on the country
The Brazil Ministry of Industry, external commerce and service releases info from what was imported/exported on the country on this site: http://comexstat.mdic.gov.br/

The data is from 1997 to 2020, and a simple analysis if Brazil import or export more jewelry can tell us if Brazil was competitive prices on the exterior. It can also tell us if jewelry, semi-jewelry or bijou has a bigger market, what country import more jewelery from Brazil, etc

# Table of Contents

- 1. Dataset and ETL
    - 1.1. Import Dataset
    - 1.2. Change to text
        - 1.2.1. Country Code
        - 1.2.2. Block Code
        - 1.2.3. Transport Via Code
    - 1.3. Drop unused columns
    - 1.4. Rename columns
    - 1.5. Rename VIA
    - 1.6. Drop 2020
- 2. Analysis
    - 2.1. Trade Balance
        - 2.1.1. Exports by Group
		- 2.1.2. Exchange Rate Correlation
		- 2.1.3. Jewelry and Similar Trade Balance
		- 2.1.4. Jewelry and Similar Exchange Rate Correlation
		- 2.1.5. Precious Jewelry VS Costume Jewelry Export
		- 2.1.6. Precious Jewelry VS Costume Jewelry Import
		- 2.1.7. Precious Jewelry Trade Balance
        - 2.1.8. Costume Jewelry Trade Balance
    - 2.2. Precious Jewelry Export Map
        - 2.2.1. Precious Jewelry Export Region
        - 2.2.2. Precious Jewelry Export to what Countrys?
- 3. Conclusion
    - 3.1. TL;DR
    - 3.2. Objective & Final Considerations

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # beautiful graphs
import matplotlib.pyplot as plt #math stuf
import datetime as dt #for date math
import re #regex
import json #json parser

## 1. Dataset and ETL
### 1.1. Import Dataset

Ok, so Brazil updates each month the export/import info posted on SIXCOMEX, the software that Brazilian companies use to declare their import/export.

They don't release all the info for each transaction, but they do release the NCM (Code description of the product - Mercosul Common Nomeclature) of the transaction.

This number describe the product been traded. Luckyly for us, jewelry and similar products all fall on the 71.XX.XX.XX code.

So let's import the exp. and imp. info and drop everything not on the 71.XX.XX.XX NCM.

In [ ]:
#Let's import the dataset
#Export
df_export = pd.read_csv('../input/brazil-exportimport-information/EXP_COMPLETA.csv', sep=';')
df_export = df_export[(df_export['CO_NCM'] >= 71000000) & (df_export['CO_NCM'] < 72000000)]
df_export = df_export.reset_index(drop=True)

#Import
df_import = pd.read_csv('../input/brazil-exportimport-information/IMP_COMPLETA.csv', sep=';')
df_import = df_import[(df_import['CO_NCM'] >= 71000000) & (df_import['CO_NCM'] < 72000000)]
df_import = df_import.reset_index(drop=True)

### 1.2. Change to text

From this dataset, i think we will probably not use the QT_ESTAT (From what i understand, is how much of the product was exported/imported).
But we will probably use all the other info.

Since a lot of that info is qualitative, let's change from number to some descritive text.

#### 1.2.1. Country Code

In [ ]:
df_country = pd.read_csv('../input/brazil-exportimport-information/PAIS.csv', sep=';', encoding='iso-8859-1')

country_dict = {}
for i, row in df_country.iterrows():
    country_dict[row['CO_PAIS']] = row['NO_PAIS_ING']

#Export
df_export['COUNTRY'] = df_export['CO_PAIS'].map(country_dict)
#Import
df_import['COUNTRY'] = df_import['CO_PAIS'].map(country_dict)

#### 1.2.2. Block Code

In [ ]:
df_block = pd.read_csv('../input/brazil-exportimport-information/PAIS_BLOCO.csv', sep=';', encoding='iso-8859-1')

block_dict = {}
for i, row in df_block.iterrows():
    block_dict[row['CO_PAIS']] = row['NO_BLOCO_ING']

#Export
df_export['BLOCK'] = df_export['CO_PAIS']
df_export['BLOCK'] = df_export['BLOCK'].map(block_dict)

#Import
df_import['BLOCK'] = df_import['CO_PAIS']
df_import['BLOCK'] = df_import['BLOCK'].map(block_dict)

#### 1.2.3. Transport Via Code

In [ ]:
df_via = pd.read_csv('../input/brazil-exportimport-information/VIA.csv', sep=';', encoding='iso-8859-1')

via_dict = {}
for i, row in df_via.iterrows():
    via_dict[row['CO_VIA']] = row['NO_VIA']

#Export
df_export['VIA'] = df_export['CO_VIA']
df_export['VIA'] = df_export['VIA'].map(via_dict)
#Import
df_import['VIA'] = df_import['CO_VIA']
df_import['VIA'] = df_import['VIA'].map(via_dict)

### 1.3. Drop unused columns

The URF Code is from what port/aeroport the product as shipped/recived. We don't care much about this info, so let's drop this column.

**QT_ESTAT** and **CO_UNID** are what type of unity is used (Box, pallet, etc) and how much of that unit was shipped. Also not a relevant info, so let's drop it.

Since we are were, let's also drop the via and country code column.

In [ ]:
#Export
df_export = df_export.drop(['QT_ESTAT', 'CO_UNID', 'CO_URF', 'CO_VIA'], axis=1)
#Import
df_import = df_import.drop(['QT_ESTAT', 'CO_UNID', 'CO_URF', 'CO_VIA'], axis=1)

### 1.4. Rename columns

In [ ]:
#Export
df_export = df_export.rename(columns = {'CO_ANO':'YEAR','CO_MES':'MONTH','CO_NCM':'NCM','SG_UF_NCM':'UF','KG_LIQUIDO':'KG'}, inplace = False)
#Import
df_import = df_import.rename(columns = {'CO_ANO':'YEAR','CO_MES':'MONTH','CO_NCM':'NCM','SG_UF_NCM':'UF','KG_LIQUIDO':'KG'}, inplace = False)

### 1.5. Rename VIA

Unfortunaly for me, the VIA.csv file doesn't come with a english translation. So i will translate by hand.

VIA is the info on what kind of method of transportation was used.

In [ ]:
via_ing_dict={
'AEREA':'Air',
'MARITIMA':'Sea',
'MEIOS PROPRIOS':'Own Means',
'RODOVIARIA':'Road',
'POSTAL':'Postal',
'VIA NAO DECLARADA':'Non Declared',
'EM MAOS':'By Hand',
'FERROVIARIA':'Rail',
'VICINAL FRONTEIRICO':'Border',
'FLUVIAL':'Fluvial',
'ENTRADA/SAIDA FICTA':'Ficta',
'COURIER':'Courier'
}

#Export
df_export['VIA'] = df_export['VIA'].map(via_ing_dict)
#Import
df_import['VIA'] = df_import['VIA'].map(via_ing_dict)

### 1.6. Drop 2020

Since we don't have the full 2020 year, it will be simpler to just drop the entire year and just do analysis with the full years that we have.

In [ ]:
#Export
df_export = df_export[df_export['YEAR'] < 2020]
#Import
df_import = df_import[df_import['YEAR'] < 2020]

## 2. Analysis

### 2.1. Trade Balance

In [ ]:
df_all = pd.concat([df_export.assign(TRADE='Export'), df_import.assign(TRADE='Import')])
df_all = df_all.reset_index(drop=True)

df_group = df_all.groupby(['YEAR','TRADE']).sum()
df_group = df_group.reset_index()

sns.set(style='darkgrid')
plt.figure(figsize=(10, 10))
ax = sns.lineplot(x='YEAR',y='VL_FOB',data=df_group,hue='TRADE')
ax.set(xlabel='Year', ylabel='US$ FOB')

Nice, this already give us a important info, Brazil export wayyyy more precius metal, gems and jewelry then import.

Let's see if this is mostly precious metals, gems or jewelry.

#### 2.1.1. Exports Group

The 71.XX.XX.XX NCM can be divided on three big groups.

* **71.01 to 71.05 - Gems, pearls, sintetic gems, etc**
* **71.06 to 71.12 - Silver, gold, platinum, etc**
* **71.13 to 71.18 - Precious jewelry, costume jewelry, coins, etc**

From those three, we also want to subdive the Jewelry and costume jewelry group.

* **71.13.1 and 71.14.1 precious metals**
* **71.13.2 and 71.14.2 metals plated with precious metal**
* **71.15 other things made with precious metal (platinum catalyst,etc)**
* **71.16 pearls and gem jewelry**
* **71.17 costume jewlery (bijou)**
* **71.18 coins**

In [ ]:
#Group
GEM_MIN = 71010000
GEM_MAX = 71059999
METAL_MIN = 71060000
METAL_MAX = 71129999
JEWELRY_MIN = 71130000
JEWELRY_MAX = 71189999

#Subgroup
PRECIOUS_MIN_1 = 71131000
PRECIOUS_MAX_1 = 71131999
PRECIOUS_MIN_2 = 71141000
PRECIOUS_MAX_2 = 71141999
PLATED_MIN_1 = 71132000
PLATED_MAX_1 = 71132999
PLATED_MIN_2 = 71142000
PLATED_MAX_2 = 71142999
OTHERS_MIN = 71150000
OTHERS_MAX = 71159999
PEARLS_GEM_MIN = 71160000
PEARLS_GEM_MAX = 71169999
BIJOU_MIN = 71170000
BIJOU_MAX = 71179999
COIN_MIN = 71180000
COIN_MAX = 71189999

In [ ]:
df_all.loc[(df_all['NCM']>=METAL_MIN) & (df_all['NCM']<=METAL_MAX),'GROUP'] = 'Metal'
df_all.loc[(df_all['NCM']>=GEM_MIN) & (df_all['NCM']<=GEM_MAX),'GROUP'] = 'Gem'
df_all.loc[(df_all['NCM']>=JEWELRY_MIN) & (df_all['NCM']<=JEWELRY_MAX),'GROUP'] = 'Jewelry'

In [ ]:
df_all.loc[(df_all['NCM']>=PRECIOUS_MIN_1) & (df_all['NCM']<=PRECIOUS_MAX_1),'SUBGROUP'] = 'Precious'
df_all.loc[(df_all['NCM']>=PRECIOUS_MIN_2) & (df_all['NCM']<=PRECIOUS_MAX_2),'SUBGROUP'] = 'Precious'
df_all.loc[(df_all['NCM']>=PLATED_MIN_1) & (df_all['NCM']<=PLATED_MAX_1),'SUBGROUP'] = 'Plated'
df_all.loc[(df_all['NCM']>=PLATED_MIN_2) & (df_all['NCM']<=PLATED_MAX_2),'SUBGROUP'] = 'Plated'
df_all.loc[(df_all['NCM']>=OTHERS_MIN) & (df_all['NCM']<=OTHERS_MAX),'SUBGROUP'] = 'Others'
df_all.loc[(df_all['NCM']>=PEARLS_GEM_MIN) & (df_all['NCM']<=PEARLS_GEM_MAX),'SUBGROUP'] = 'Pearls & Gems'
df_all.loc[(df_all['NCM']>=BIJOU_MIN) & (df_all['NCM']<=BIJOU_MAX),'SUBGROUP'] = 'Bijou'
df_all.loc[(df_all['NCM']>=COIN_MIN) & (df_all['NCM']<=COIN_MAX),'SUBGROUP'] = 'Coin'

In [ ]:
df_group = df_all.groupby(['YEAR','TRADE','GROUP']).sum()
df_group = df_group.reset_index()
df_group = df_group[df_group['TRADE']=='Export']
df_group = df_group.drop(columns=['MONTH','NCM','KG','TRADE'])

df_plot = df_group.pivot(columns='GROUP', index='YEAR', values='VL_FOB')

plt.figure(figsize=(10, 10))
ax.set(xlabel='Year', ylabel='US$ FOB')
df_plot.plot(kind='bar', stacked=True)

Uhhhg...As is standard in my country, we export comodities, not finished products.

While we export 2-4 billion dolars in precious metal every year since 2010, we export only 100-200 million of finished products.

#### 2.1.2. Exchange Rate Correlation

Does this grownth in export correlate to the growth of the exchange rate of USD/BRL coin?

In [ ]:
df_exchange = pd.read_csv('../input/currency-exchange-rate-usdbrl-19932019/Month.csv',dayfirst=True,parse_dates=[0])
df_exchange['Mean'] = (df_exchange['Max'] + df_exchange['Min'])/2

df_exchange = df_exchange.groupby(df_exchange['Date'].map(lambda x: x.year)).mean()
df_exchange = df_exchange.reset_index()

df_group = df_all.groupby(['YEAR','TRADE']).sum()
df_group = df_group.reset_index()

df_exp_exchange = df_group[df_group['TRADE']=='Export']
df_exp_exchange = df_exp_exchange.merge(df_exchange,left_on='YEAR',right_on='Date')
df_exp_exchange = df_exp_exchange[['VL_FOB','Mean']]
df_exp_exchange = (df_exp_exchange-df_exp_exchange.min())/(df_exp_exchange.max()-df_exp_exchange.min())

print(df_exp_exchange.corr(method='pearson'))

sns.set(style='darkgrid')
plt.figure(figsize=(10, 10))
ax.set(ylabel='EXP Value', xlabel='BRL/USD Exchange')
ax = sns.scatterplot(y='VL_FOB', x='Mean', data=df_exp_exchange)

As we can see, there seens to be some correlation beteween the high value of dolar when compared to real, but only p=0.480602 wich is not a strong correlation.

#### 2.1.3. Jewelry and Similar Trade Balance

Let's see the trade balance for only the group of manufactured goods

**71.13 to 71.18** - Jewelry, bijou, coins, etc

In [ ]:
df_group = df_all.groupby(['YEAR','TRADE','GROUP']).sum()
df_group = df_group.reset_index()
df_group = df_group[df_group['GROUP']=='Jewelry']

sns.set(style='darkgrid')
plt.figure(figsize=(10, 10))
ax.set(xlabel='Year', ylabel='US$ FOB')
ax = sns.lineplot(x='YEAR',y='VL_FOB',data=df_group,hue='TRADE')

Well, considering only manufactured goods, we have a a more equal trade balance.
We export more than import, but not order of magnitudes more.

#### 2.1.4. Jewelry and Similar Exchange Rate Correlation

In [ ]:
df_group = df_all.groupby(['YEAR','TRADE','GROUP']).sum()
df_group = df_group.reset_index()
df_group = df_group[df_group['GROUP']=='Jewelry']
df_exp_exchange = df_group[df_group['TRADE']=='Export']
df_exp_exchange = df_exp_exchange[df_exp_exchange['GROUP']=='Jewelry']

df_exp_exchange = df_exp_exchange.merge(df_exchange,left_on='YEAR',right_on='Date')
df_exp_exchange = df_exp_exchange[['VL_FOB','Mean']]
df_exp_exchange = (df_exp_exchange-df_exp_exchange.min())/(df_exp_exchange.max()-df_exp_exchange.min())

print(df_exp_exchange.corr(method='pearson'))

sns.set(style='darkgrid')
plt.figure(figsize=(10, 10))
ax.set(ylabel='EXP Value', xlabel='BRL/USD Exchange')
ax = sns.scatterplot(y='VL_FOB', x='Mean', data=df_exp_exchange)

So, probably no correlation between the BRL/USD exchange and export value.
Which means that the export of jewelry is probably stable and non dependent on the dolar value.

#### 2.1.5. Precious Jewelry VS Costume Jewelry Export

Ok, now let's how the export of the subgroup of jewelry is distribuited.
We have Costume Jewelry (Bijou), Pearls & Gems Jewelry, Plated Jewelry, Precious Jewelry,Coins and Others.

We don't care about Coin and Others, so let's drop that.

In [ ]:
df_plot = df_all.groupby(['YEAR','TRADE','SUBGROUP']).sum()
df_plot = df_plot.reset_index()
df_plot = df_plot[df_plot['TRADE']=='Export']
df_plot = df_plot[df_plot['SUBGROUP']!='Coin']
df_plot = df_plot[df_plot['SUBGROUP']!='Others']
df_plot = df_plot.pivot(columns='SUBGROUP', index='YEAR', values='VL_FOB')
plt.figure(figsize=(10, 10))
df_plot.plot(kind='bar', stacked=True)

Nice, we have a steady export of Precious Jewelry, and some export of Costume Jewelry (Bijou).

#### 2.1.6. Precious Jewelry VS Costume Jewelry Import

What about the import?

In [ ]:
df_plot = df_all.groupby(['YEAR','TRADE','SUBGROUP']).sum()
df_plot = df_plot.reset_index()
df_plot = df_plot[df_plot['TRADE']=='Import']
df_plot = df_plot[df_plot['SUBGROUP']!='Coin']
df_plot = df_plot[df_plot['SUBGROUP']!='Others']
df_plot = df_plot.pivot(columns='SUBGROUP', index='YEAR', values='VL_FOB')
plt.figure(figsize=(10, 10))
df_plot.plot(kind='bar', stacked=True)

As expected, we import a lot of Bijou. We also import a lot of Precious Jewelry.

#### 2.1.7. Precious Jewelry Trade Balance

Let's see how the trade balance for the subgroup of Precious Jewelry behaves.

In [ ]:
df_plot = df_all.groupby(['YEAR','TRADE','SUBGROUP']).sum()
df_plot = df_plot.reset_index()
df_plot = df_plot[df_plot['SUBGROUP']=='Precious']
#df_plot['VL_FOB'] = np.where(df_plot.TRADE=='Import',-df_plot.VL_FOB,df_plot.VL_FOB)

sns.set(style='darkgrid')
plt.figure(figsize=(10, 10))
ax.set(xlabel='Year', ylabel='US$ FOB')
ax = sns.lineplot(x='YEAR',y='VL_FOB',data=df_plot,hue='TRADE')

Ok, so until 2013 we had exported more precious jewelry then imported, but it seens that trend is passing in 2019.

#### 2.1.8. Costume Jewelry Trade Balance

What about bijou?

In [ ]:
df_plot = df_all.groupby(['YEAR','TRADE','SUBGROUP']).sum()
df_plot = df_plot.reset_index()
df_plot = df_plot[df_plot['SUBGROUP']=='Bijou']
#df_plot['VL_FOB'] = np.where(df_plot.TRADE=='Import',-df_plot.VL_FOB,df_plot.VL_FOB)

sns.set(style='darkgrid')
plt.figure(figsize=(10, 10))
ax.set(xlabel='Year', ylabel='US$ FOB')
ax = sns.lineplot(x='YEAR',y='VL_FOB',data=df_plot,hue='TRADE')

As expected, we import more costume jewelry than export.


### 2.2. Precious Jewelry Export Map

What state from Brazil export more Jewelry? And to what countries?

We can use [Kepler.gl](https://kepler.gl/), a open souce geospatial analysis tool made by uber and generate some cools maps and check that.

#### 2.2.1. Jewelry Export Region

Let's see what states from Brazil export more Jewelry.

In [ ]:
!pip install ipywidgets
!pip install keplergl
!jupyter nbextension install --py --sys-prefix keplergl
!jupyter nbextension enable keplergl --py --sys-prefix

In [ ]:
df_map = df_all.groupby(['UF','TRADE','SUBGROUP']).sum()
df_map = df_map.reset_index()
df_map = df_map[df_map['SUBGROUP']=='Precious']
df_map = df_map[df_map['TRADE']=='Export']
df_map = df_map.drop(columns=['NCM','KG','TRADE','SUBGROUP','YEAR','MONTH','CO_PAIS'])
df_map = df_map.reset_index(drop=True)

#We removed some UFs when we selected only rows with the subgroup 'Precious'
#For the map we need to put then back
df_uf = pd.read_csv('../input/brazil-exportimport-information/UF.csv', sep=';', encoding='iso-8859-1')
for i, row in df_uf.iterrows():
    if (df_map['UF'].str.contains(row['SG_UF']).any()):
        df_map.loc[(df_map['UF']==row['SG_UF']),'UF_NAME'] = row['NO_UF']
    else:
        new_row = {'UF':row['SG_UF'], 'VL_FOB':0, 'UF_NAME':row['NO_UF']}
        df_map = df_map.append(new_row, ignore_index=True)
#Drop UF's code that don't correlate to a State
df_map = df_map[~df_map['UF'].str.contains('EX|CB|MN|ND|RE|ZN|ED')]

In [ ]:
brazil_geo = json.load(open('../input/brazil-geojson/brazil_geo.json'))
brazil_geo = brazil_geo['features']

uf_geo = {}

for uf in brazil_geo:
    uf_geo[uf['id']] = uf['geometry']
    
df_map['GEOMETRY'] = df_map['UF'].map(uf_geo)

In [ ]:
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "elzzyyt",
          "type": "geojson",
          "config": {
            "dataId": "dj3gzji39",
            "label": "Export Total (1997 to 2019)",
            "color": [
              255,
              203,
              153
            ],
            "columns": {
              "geojson": "GEOMETRY"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 1,
              "strokeOpacity": 0.8,
              "thickness": 1,
              "strokeColor": [
                38,
                26,
                16
              ],
              "colorRange": {
                "name": "ColorBrewer YlGn-8",
                "type": "sequential",
                "category": "ColorBrewer",
                "colors": [
                  "#ffffe5",
                  "#f7fcb9",
                  "#d9f0a3",
                  "#addd8e",
                  "#78c679",
                  "#41ab5d",
                  "#238443",
                  "#005a32"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": True,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "VL_FOB",
              "type": "integer"
            },
            "colorScale": "quantile",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "dj3gzji39": [
              {
                "name": "VL_FOB",
                "format": None
              },
              {
                "name": "UF",
                "format": None
              },
              {
                "name": "UF_NAME",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": -14.927493161931578,
      "longitude": -56.124965564305676,
      "pitch": 0,
      "zoom": 2.7402945845847593,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": False,
        "road": False,
        "border": True,
        "building": False,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

^ There is a bunch of code here to config how kepler shows the map, if you want to use kepler, i recomend you get the .csv of your dataframe, upload to the site, do the changes to the map there and then copy this part from the site.

In [ ]:
from keplergl import KeplerGl

map_export_total = KeplerGl()
map_export_total.add_data(data=df_map, name='dj3gzji39')
map_export_total.config = config

map_export_total

As we can see, the states that export more are the expected ones. Brazil has more industry and a bigger economy on the south and southeast states.

And we have a kind of duty free zone on Manaus (Amazonia State), wich also seen's to export a lot of jewelry.

#### 2.2.2. Jewelry Export to what Countrys?

Let's see what country imported the most jewelry from Brazil.

In [ ]:
df_map = df_all.groupby(['COUNTRY','TRADE','SUBGROUP']).sum()
df_map = df_map.reset_index()
df_map = df_map[df_map['SUBGROUP']=='Precious']
df_map = df_map[df_map['TRADE']=='Export']
df_map = df_map.drop(columns=['TRADE','SUBGROUP','YEAR','MONTH','NCM','CO_PAIS','KG'])
df_map = df_map.sort_values(by=['VL_FOB'],ascending=False)
df_map = df_map.reset_index(drop=True)

In [ ]:
df_country = pd.read_csv('../input/world-capitals-gps/concap.csv', sep=',', encoding='iso-8859-1')
df_country

df_map = df_map.merge(df_country,left_on='COUNTRY',how='left',right_on='CountryName',suffixes=('_map', '_country'))

In [ ]:
df_map = df_map.drop(columns=['CountryName','CapitalName','CountryCode','ContinentName'])
df_map[df_map['CapitalLatitude'].isna()]

df_map_missing = pd.DataFrame([
    ['Bahrein', 18314598, 26.216667, 50.583333],
    ['French Guyana', 264708, 18.35, -64.933333],
    ['Virgin Islands (USA)', 229499, 18.333333, -64.833333],
    ['Netherlands Antilles', 131703, 12.226079 -69.060087],
    ["Cote D'Ivore", 53217, 6.85, -5.3],
    ['Lebuan', 30034, 5.315894, 115.219033],
    ['Guadeloupe', 21850, 16.270000, -61.580002],
    ['Reunion', 19821, -21.114444, 55.5325],
    ['Martinique', 8856, 14.666667, -61],
    ['Brunei', 230, 4.890283, 114.942217],
    ['Hong Kong', 4441562, 22.3, 114.2],
], columns=['COUNTRY','VL_FOB','CapitalLatitude','CapitalLongitude'])

df_map = df_map[df_map['CapitalLatitude']!=0]
df_map = df_map.append(df_map_missing)
df_map = df_map.dropna(axis='rows')
df_map = df_map.sort_values(by=['VL_FOB'],ascending=False)
df_map = df_map.reset_index(drop=True)

So, the top 20 importers of Brazilian Jewlery are as follows:

In [ ]:
df_map.head(20)

What country is [Bahrein](https://en.wikipedia.org/wiki/Bahrain)???

Aparently is a realy small country on the Persian Gulf.
To give you a picture of how small is the country, they have a total area of 780km² (São Paulo, the biggest city in Brazil is 1.521km²) and has a population of 1.569.446 (São Paulo has 12.252.023).

And a small country like that is the 7º importer of Brazilian Jewelry.

In [ ]:
config = {
  "version": "v1",
  "config": {
    "visState": {
      "filters": [],
      "layers": [
        {
          "id": "a8vgbvm",
          "type": "point",
          "config": {
            "dataId": "cmt54a6kj",
            "label": "Export Total (1997 to 2019)",
            "color": [
              118,
              183,
              61
            ],
            "columns": {
              "lat": "CapitalLatitude",
              "lng": "CapitalLongitude",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 10,
              "fixedRadius": False,
              "opacity": 0.8,
              "outline": False,
              "thickness": 2,
              "strokeColor": None,
              "colorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radiusRange": [
                0,
                150
              ],
              "filled": True
            },
            "hidden": False,
            "textLabel": []
          },
          "visualChannels": {
            "colorField": None,
            "colorScale": "quantile",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "sizeField": {
              "name": "VL_FOB",
              "type": "integer"
            },
            "sizeScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "cmt54a6kj": [
              {
                "name": "COUNTRY",
                "format": None
              },
              {
                "name": "VL_FOB",
                "format": None
              }
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": 8.884265547401553,
      "longitude": -19.06778798832715,
      "pitch": 0,
      "zoom": 0.8934685912520477,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": False,
        "road": False,
        "border": True,
        "building": False,
        "water": True,
        "land": False,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [ ]:
map_export_world = KeplerGl()
map_export_world.add_data(data=df_map, name='cmt54a6kj')
map_export_world.config = config

map_export_world

## 3. Conclusion

### 3.1. TL;DR

- We did some cools maps with Kepler.gl
- Brazil export a lot of commodities related to jewelry (Gold, silver, gems)
- Brazil import more costume jewelry (bijou) than export
- Brazil export jewelry principally to EUA, Switzerland, Israel, Peru and Argentina
- Brazil seens to export more then import jewelry, but that changed recently

### 3.2. Objective & Final Considerations

We used kepler.gl to make some maps, and got some inshigths on Brazilian commerce of jewelry.